# Entendendo o Problema

Uma imobiliária situada na cidade do Rio de Janeiro está enfrentando dificuldades para alugar e vender imóveis. Durante uma pesquisa sobre como empresas semelhantes operam no mercado, a imobiliária identificou que esse problema pode estar relacionado aos valores dos imóveis e às recomendações feitas aos clientes.

## Objetivo do Projeto

Esse projeto tem como objetivo solucionar os desafios enfrentados pela imobiliária por meio de três etapas principais:

1. **Análise e Tratamento dos Dados**  
   - Ler e realizar o tratamento do histórico de preços de imóveis no Rio de Janeiro.  
   - Identificar padrões e tendências que podem estar influenciando o mercado.  

2. **Construção de um Modelo de Regressão**  
   - Desenvolver um modelo de regressão para precificar imóveis de forma precisa e competitiva.  

3. **Criação de um Sistema Recomendador**  
   - Construir um recomendador de imóveis para oferecer sugestões personalizadas aos clientes com base em suas preferências e perfil.



# Extraindo base de dados

In [1]:
!wget https://caelum-online-public.s3.amazonaws.com/challenge-spark/semana-1.zip && unzip semana-1.zip

--2024-11-28 19:46:36--  https://caelum-online-public.s3.amazonaws.com/challenge-spark/semana-1.zip
Resolving caelum-online-public.s3.amazonaws.com (caelum-online-public.s3.amazonaws.com)... 52.216.218.73, 3.5.30.39, 52.216.214.137, ...
Connecting to caelum-online-public.s3.amazonaws.com (caelum-online-public.s3.amazonaws.com)|52.216.218.73|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18975214 (18M) [application/zip]
Saving to: ‘semana-1.zip’

semana-1.zip        100%[===================>]  18.10M  6.81MB/s    in 2.7s    

2024-11-28 19:46:40 (6.81 MB/s) - ‘semana-1.zip’ saved [18975214/18975214]

Archive:  semana-1.zip
  inflating: dataset_bruto.json      


# Instalando as dependências

In [2]:
!pip install -q findspark
import findspark
findspark.init()

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.types import IntegerType, DoubleType, FloatType
from pyspark.sql import functions as f
from pyspark.ml.feature import VectorAssembler, StandardScaler, PCA
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator, ClusteringEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.linalg import Vectors
from pyspark.ml.clustering import KMeans
from pyspark.ml import Pipeline
from scipy.spatial.distance import euclidean
import numpy as np

# Inicializando Sessão Spark

In [4]:
spark = SparkSession.builder \
  .master('local[*]') \
  .appName("previsao_imoveis") \
  .getOrCreate()

In [5]:
df = spark.read.json('/content/dataset_bruto.json')

In [6]:
df.show(5)

+--------------------+--------------------+--------------------+
|             anuncio|             imagens|             usuario|
+--------------------+--------------------+--------------------+
|{0, [], [16], [0]...|[{39d6282a-71f3-4...|{9d44563d-3405-4e...|
|{0, [], [14], [0]...|[{23d2b3ab-45b0-4...|{36245be7-70fe-40...|
|{0, [1026], [1026...|[{1da65baa-368b-4...|{9dc415d8-1397-4d...|
|{0, [120], [120],...|[{79b542c6-49b4-4...|{9911a2df-f299-4a...|
|{0, [3], [3], [0]...|[{e2bc497b-6510-4...|{240a7aab-12e5-40...|
+--------------------+--------------------+--------------------+
only showing top 5 rows



Para nossa análise, apenas as informações do campo "anuncio" serão relevantes.

In [7]:
analise = df.select('anuncio.*')

# Tratamento e preparando os dados

Antes de criar a função, analisei e entendi que a base de dados necessitava do seguinte tratamento:

1.   Transformação dos dados das colunas "quartos", "suites", "banheiros", "vaga", "area_total" e "area_util" de listas para inteiros.
2.   Transformação dos dados da coluna "valores" em colunas separadas para melhor compreensão.
3.   A equipe solicitou que apenas as informações sobre bairro e zona da cidade fossem extraídas, como se trata de um estudo sobre o preço de venda dos imóveis, será utilizada apenas as informações do tipo VENDA, também foi solicitado que fizéssemos alguns filtros nas colunas tipo_uso, tipo_unidade e tipo_anuncio da nossa base de dados: tipo_uso: Residencial, tipo_unidade: Apartamento, tipo_anuncio: Usado e deveria apagar a coluna "area_total" pois tem as mesmas informações de "area_util" e mais valores ausentes.
4. Tratamento coluna "características" e se a lista for vazia retornar 'nao informado'.
5. Transformação dados NA em 0.
6. Converter o tipo de colunas numéricas, como "andar", "banheiros", "suites" e "quartos" para o tipo inteiro e converter as colunas "area_util", "condominio", "iptu" e "valor" para o tipo double.
7. Transformar variáveis categóricas em binárias.
8. Um parametro para diferenciar se o tratamento é para o modelo de regressão ou para o recomendador.

In [8]:
def tratar_dados(df, modelo):
    # 1. Converte colunas de listas para inteiros
    colunas_a_transformar = ['area_total', 'quartos', 'suites', 'banheiros', 'vaga', 'area_util']
    for coluna in colunas_a_transformar:
        df = df.withColumn(coluna, f.element_at(f.col(coluna), 1).cast(IntegerType()))

    # 2. Explode a coluna "valores" e extrai informações relevantes
    df = df.withColumn('valores', f.explode('valores')).select(
        '*',
        f.col('valores.condominio').alias('condominio'),
        f.col('valores.iptu').alias('iptu'),
        f.col('valores.tipo').alias('tipo'),
        f.col('valores.valor').alias('valor'),
        f.col('endereco.bairro').alias('bairro'),
        f.col('endereco.zona').alias('zona')
    )

    # 3. Filtra os dados conforme os critérios
    df = df.filter(
        (f.col('tipo_uso') == 'Residencial') &
        (f.col('tipo_anuncio') == 'Usado') &
        (f.col('tipo_unidade') == 'Apartamento') &
        (f.col('tipo') == 'Venda')
    ).drop('valores', 'endereco', 'tipo_uso', 'tipo_unidade', 'tipo_anuncio', 'area_total', 'tipo')

    # 4. Trata a coluna "caracteristicas"
    df = df.withColumn(
        'caracteristicas',
        f.when(f.size(f.col('caracteristicas')) == 0, f.array(f.lit('nao informado'))).otherwise(f.col('caracteristicas'))
    )

    # 5. Preenche valores nulos com 0
    df = df.na.fill({'iptu': 0, 'quartos': 0, 'suites': 0, 'banheiros': 0, 'vaga': 0, 'condominio': 0})

    # 6. Converte tipos de colunas
    colunas_para_conversao = {
        'andar': IntegerType(),
        'banheiros': IntegerType(),
        'suites': IntegerType(),
        'quartos': IntegerType(),
        'area_util': DoubleType(),
        'condominio': DoubleType(),
        'iptu': DoubleType(),
        'valor': DoubleType()
    }
    for coluna, tipo in colunas_para_conversao.items():
        df = df.withColumn(coluna, f.col(coluna).cast(tipo))

    # 7. Cria colunas binárias para características
    caracteristicas = [
        'Academia', 'Churrasqueira', 'Playground', 'Condomínio fechado',
        'Portão eletrônico', 'Portaria 24h', 'Salão de festas',
        'Piscina', 'Animais permitidos', 'Elevador'
    ]
    for caracteristica in caracteristicas:
        df = df.withColumn(
            f"caracteristica_{caracteristica.replace(' ', '_')}",
            f.array_contains(f.col('caracteristicas'), caracteristica).cast("int")
        )

    # Cria colunas binárias para zonas
    zonas = ['Zona Norte', 'Zona Oeste', 'Zona Central', 'Zona Sul']
    for zona in zonas:
        df = df.withColumn(
            f"{zona.replace(' ', '_')}",
            (f.col('zona') == zona).cast("int")
        )

    # 8. Remove colunas desnecessárias de acordo com o modelo
    colunas_a_remover = ['caracteristicas', 'zona']
    if modelo != 'rec':
        colunas_a_remover.extend(['id', 'bairro'])
    df = df.drop(*colunas_a_remover)

    return df


A conversão de tipos de colunas pode parecer redundante em alguns trechos, mas isso acontece porque há duas etapas distintas no processo, cada uma com uma motivação diferente:

**1. Conversão inicial de colunas de listas para inteiros (`f.element_at`)**

Na etapa inicial, algumas colunas como `"area_total"`, `"quartos"`, etc., estão representadas como listas (ou arrays) e precisam ser extraídas. O método `f.element_at(f.col(coluna), 1)` pega o primeiro elemento da lista e o converte para um tipo numérico (neste caso, `IntegerType`).

Isso é necessário porque:
- Esses valores chegam como listas devido à forma como os dados são estruturados originalmente (possivelmente em JSON ou outro formato de aninhamento).
- Apenas extrair o valor do array não garante que o tipo da coluna seja o desejado, já que Spark pode inferir um tipo diferente.

**2. Conversão geral de tipos de colunas para consistência**

Mais tarde, existe uma conversão explícita de várias colunas (como `"andar"`, `"quartos"`, `"area_util"`, etc.) para tipos como `IntegerType` ou `DoubleType`.

Essa conversão ocorre por dois motivos:
1. **Correção de inconsistências nos dados**: Mesmo após o tratamento inicial, algumas colunas podem ter valores nulos ou tipos inesperados devido a transformações anteriores ou ao carregamento dos dados.
2. **Normalização do tipo para o modelo ou análise**: Certos algoritmos de machine learning ou análises requerem tipos específicos (ex.: números inteiros ou de ponto flutuante). Essa conversão garante que os dados estejam no formato correto.

**Por que não fazer tudo de uma vez?**

Não é possível consolidar essas etapas em uma única conversão porque:
- A primeira conversão (`f.element_at`) transforma dados do tipo lista/array em valores simples. Isso deve ser feito antes de qualquer conversão de tipo final.
- A segunda conversão atua em colunas que já passaram por diversos tratamentos, incluindo preenchimento de valores nulos e filtragem.

Ao final, cada conversão atende a uma necessidade específica e é aplicada no momento adequado. Se o dado já viesse normalizado, a primeira conversão seria desnecessária.

In [9]:
modelo = tratar_dados(analise, 'reg')

In [10]:
# 1. Identificação das Features
features_col = [col for col in modelo.columns if col != 'valor']

# 2. Assembler
vector_assembler = VectorAssembler(inputCols=features_col, outputCol="features")

# 3. Transformação
modelo_vectorizado = vector_assembler.transform(modelo)

# 4. Seleção das Colunas Relevantes
modelo_vectorizado = modelo_vectorizado.select("features", "valor")

# 5. Exibir o DataFrame vetorizado
modelo_vectorizado.show(truncate=False)

+-----------------------------------------------------------------------------------------------+-------+
|features                                                                                       |valor  |
+-----------------------------------------------------------------------------------------------+-------+
|[3.0,43.0,1.0,2.0,0.0,1.0,245.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0]   |15000.0|
|(22,[0,1,2,3,5,9,10,11,12,13,14,16,19],[2.0,42.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0]) |15000.0|
|(22,[0,1,2,3,5,9,10,11,12,13,14,19],[1.0,41.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])        |20000.0|
|[3.0,43.0,1.0,2.0,0.0,0.0,285.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0]   |20000.0|
|[2.0,43.0,1.0,2.0,0.0,1.0,245.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0]   |15000.0|
|[3.0,43.0,1.0,2.0,0.0,0.0,285.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0]   |20000.0|
|[3.0,43.0,1.0,2.0,0.0,1.0,250.0,0.0,0.0,1.0,1

1. **Identificação das Features**:
   - As colunas do DataFrame são analisadas, e todas as colunas exceto a variável `valor` (o alvo da previsão) são selecionadas para formar a lista de features. Essas colunas representam as variáveis preditoras que serão usadas no modelo.

2. **VectorAssembler**:
   - Um transformador do Spark que combina múltiplas colunas de entrada em uma única coluna chamada `features`. Essa coluna contém um vetor que representa todos os valores das variáveis preditoras em cada linha do DataFrame.

3. **Transformação**:
   - O `VectorAssembler` é aplicado ao DataFrame, criando a nova coluna `features`. Este vetor de preditores será usado diretamente pelos algoritmos de machine learning do Spark.

4. **Seleção das Colunas Relevantes**:
   - Após a vetorização, apenas duas colunas são mantidas:
     - `features`: o vetor das variáveis preditoras.
     - `valor`: a variável-alvo (target).

5. **Visualização**:
   - O DataFrame resultante é exibido para inspecionar os vetores de features e seus valores associados no target.

Essa etapa organiza e prepara os dados no formato esperado pela maioria dos algoritmos de machine learning do Spark, otimizando o pipeline de aprendizado.

In [11]:
# 1. Separação dos dados em treino (80%) e teste (20%)
treino, teste = modelo_vectorizado.randomSplit([0.8, 0.2], seed=42)

# 2. Exibir a contagem de registros em cada conjunto
print(f"Número de registros em treino: {treino.count()}")
print(f"Número de registros em teste: {teste.count()}")

Número de registros em treino: 53107
Número de registros em teste: 13455


1. **Separação dos Dados**:
   - O método `randomSplit` do PySpark é usado para dividir os dados em dois subconjuntos: **treino** e **teste**.
   - O argumento `[0.8, 0.2]` especifica as proporções de divisão:
     - 80% dos dados vão para o conjunto de **treino** (usado para ajustar o modelo).
     - 20% dos dados vão para o conjunto de **teste** (usado para avaliar o desempenho do modelo).
   - O parâmetro `seed=42` garante que a divisão seja **reprodutível** (a mesma divisão será gerada sempre que o código for executado com os mesmos dados).

2. **Contagem de Registros**:
   - Após a separação, os métodos `treino.count()` e `teste.count()` contam o número de registros em cada conjunto.
   - Essas contagens ajudam a verificar se os dados foram divididos nas proporções corretas.


**Importância dessa etapa:**
- A divisão em treino e teste é um **passo fundamental** no processo de aprendizado de máquina:
  - **Treino**: O modelo aprende os padrões dos dados.
  - **Teste**: O modelo é avaliado em dados que ele não viu antes, permitindo medir sua capacidade de generalizar.
- Garantir proporções balanceadas e reprodutibilidade é essencial para obter uma avaliação confiável do modelo.

# Criação do Modelo Regressor

In [12]:
# 1. Criação do modelo de regressão com Random Forest
rfr = RandomForestRegressor(seed=101, maxDepth=10, numTrees=20, featuresCol='features', labelCol='valor')

# 2. Criação do avaliador de regressão
evaluator = RegressionEvaluator(labelCol='valor', predictionCol='prediction')

# 3. Treinamento do modelo
modelo_rfr = rfr.fit(treino)

# 4. Realização de previsões
previsoes_rfr_test = modelo_rfr.transform(teste)

importancias = modelo_rfr.featureImportances.toArray()

# 6. Associar com os nomes das features
features_importancia = list(zip(features_col, importancias))

# 7. Ordenar pela importância
ordem_importancia = sorted(features_importancia, key=lambda x: x[1], reverse=True)

# 8. Exibição das features mais importantes
print('Features Mais Importantes:')
for feature, importancias in ordem_importancia:
    print(f"{feature}: {importancias}")

Features Mais Importantes:
area_util: 0.297353318864422
condominio: 0.24138059616461077
quartos: 0.1340032579632543
vaga: 0.07370743804995758
banheiros: 0.06666138273769011
Zona_Sul: 0.0469779409281179
suites: 0.04029684752861563
iptu: 0.03885480793047876
Zona_Oeste: 0.015094482370555187
andar: 0.01173371684217761
Zona_Norte: 0.011665175116091175
caracteristica_Piscina: 0.0030655559685234335
caracteristica_Portaria_24h: 0.002977592127417346
caracteristica_Academia: 0.0025648322786988044
caracteristica_Churrasqueira: 0.00243103676839295
caracteristica_Playground: 0.0022303839169130466
caracteristica_Animais_permitidos: 0.0020873561725031522
caracteristica_Portão_eletrônico: 0.0018076427956220556
caracteristica_Salão_de_festas: 0.0017666316616361689
caracteristica_Elevador: 0.0016473692963103934
caracteristica_Condomínio_fechado: 0.0015853745669736265
Zona_Central: 0.00010725995103795895


1. **Criação do modelo de regressão com Random Forest**:
   - O **RandomForestRegressor** é um modelo de machine learning utilizado para tarefas de regressão. Ele cria várias árvores de decisão e as combina para fazer previsões mais robustas.
   - O modelo foi configurado com parâmetros como o número de árvores na floresta (`numTrees=20`), a profundidade máxima das árvores (`maxDepth=10`), e as colunas que contêm as variáveis preditoras e o valor alvo (`featuresCol` e `labelCol`).

2. **Criação do avaliador de regressão**:
   - O **RegressionEvaluator** é usado para avaliar a qualidade do modelo. Ele compara as previsões feitas pelo modelo com os valores reais (coluna `valor`) e pode calcular métricas como o erro quadrático médio (RMSE) ou o R².
   - O avaliador foi configurado para usar a coluna `valor` como a variável-alvo e a coluna `prediction` como a variável com as previsões feitas pelo modelo.

3. **Treinamento do modelo**:
   - O modelo (`rfr`) foi treinado usando o conjunto de dados de treinamento (`treino`) com o método `fit()`. Esse processo ajusta o modelo aos dados, aprendendo as relações entre as features e o valor a ser predito.

4. **Realização de previsões**:
   - Após o treinamento, o modelo fez previsões no conjunto de teste (`teste`) com o método `transform()`. As previsões geradas pelo modelo são armazenadas na coluna `prediction` do DataFrame resultante.

5. **Extração das importâncias das features**:
   - Uma vez que o modelo está treinado, é possível acessar a importância de cada feature usando o atributo `featureImportances`. Isso retorna um vetor que indica o peso de cada feature na decisão das árvores de regressão do Random Forest.

6. **Associação das importâncias com os nomes das features**:
   - As importâncias das features são combinadas com seus respectivos nomes usando a função `zip()`, formando uma lista de tuplas onde cada tupla contém o nome da feature e a sua importância correspondente.

7. **Ordenação das features pela importância**:
   - A lista de features e suas importâncias é ordenada de forma decrescente para que as features mais importantes fiquem no topo.

8. **Exibição das features mais importantes**:
   - Finalmente, o código percorre a lista ordenada de features e exibe o nome de cada uma junto com seu valor de importância. Isso permite visualizar quais variáveis têm maior influência na previsão do modelo.


# Otimização e Resultados

In [13]:
rfr = RandomForestRegressor(featuresCol='features', labelCol='valor')
# 1. Construção da Grade de Parâmetros para Busca (Grid Search)
grid = ParamGridBuilder() \
    .addGrid(rfr.numTrees, [10, 20, 30]) \
    .addGrid(rfr.maxDepth, [5, 10]) \
    .addGrid(rfr.maxBins, [10, 32, 45]) \
    .build()

evaluator = RegressionEvaluator(labelCol='valor', predictionCol='prediction', metricName='rmse')

# 2. Treinamento do Modelo com Validação Cruzada
rfr_cv = CrossValidator(
    estimator=rfr,
    estimatorParamMaps=grid,
    evaluator=evaluator,
    numFolds=3
)

modelo_rfr_cv = rfr_cv.fit(treino)
previsoes_rfr_cv_teste = modelo_rfr_cv.transform(teste)

# 3. Avaliação do Modelo sem Validação Cruzada e com Validação Cruzada
print('Sem Cross Validation')
print('='*30)
print('R²: %f' % evaluator.evaluate(previsoes_rfr_test, {evaluator.metricName: 'r2'}))
print('RMSE: %f' % evaluator.evaluate(previsoes_rfr_test, {evaluator.metricName: 'rmse'}))
print('='*30)
print('='*30)
print('Com Cross Validation')
print('='*30)
print('R²: %f' % evaluator.evaluate(previsoes_rfr_cv_teste, {evaluator.metricName: 'r2'}))
print('RMSE: %f' % evaluator.evaluate(previsoes_rfr_cv_teste, {evaluator.metricName: 'rmse'}))

Sem Cross Validation
R²: 0.828595
RMSE: 601843.056074
Com Cross Validation
R²: 0.841141
RMSE: 579399.624267


1. **Construção da Grade de Parâmetros para Busca (Grid Search)**:
   - **ParamGridBuilder** cria uma grade de parâmetros a ser testada. O código define diferentes valores para três parâmetros do modelo:
     - `numTrees`: número de árvores na floresta (opções: 10, 20, 30).
     - `maxDepth`: profundidade máxima das árvores (opções: 5, 10).
     - `maxBins`: número máximo de bins (intervalos) usados para dividir os dados (opções: 10, 32, 45).
   - O objetivo é testar essas combinações de parâmetros para encontrar a configuração que gera o melhor desempenho no modelo.

2. **Validação Cruzada (Cross-Validation)**:
   - **CrossValidator** é utilizado para realizar validação cruzada no modelo. Ele divide o conjunto de dados de treino em 3 partes (folds), treina o modelo 3 vezes (uma para cada divisão) e testa em cada uma das partes separadas.
   - A validação cruzada ajuda a reduzir o risco de overfitting, fornecendo uma avaliação mais robusta do modelo.

3. **Avaliação do Modelo sem Validação Cruzada e com Validação Cruzada**:
   - O modelo de Random Forest, sem validação cruzada, é avaliado com base no **R²** (coeficiente de determinação) e **RMSE** para as previsões feitas no conjunto de teste e o modelo treinado com validação cruzada é avaliado com as mesmas métricas (**R²** e **RMSE**) nas previsões feitas no conjunto de teste, permitindo comparar como o modelo se comporta com e sem a validação cruzada.

Em resumo, essa etapa treina e avalia um modelo de Random Forest de duas maneiras: sem validação cruzada e com validação cruzada, e compara os resultados usando as métricas de avaliação de regressão.


### Sem Validação Cruzada
- **R²: 0.828595**: O coeficiente de determinação (R²) indica que aproximadamente 82.86% da variância nos dados de saída pode ser explicada pelo modelo. Isso sugere que o modelo tem um bom ajuste aos dados.
- **RMSE: 601843.056074**: O erro quadrático médio (RMSE) mede a diferença média entre os valores previstos pelo modelo e os valores reais. Um RMSE de 601843.056074 indica que, em média, as previsões do modelo estão a cerca de 601843 unidades do valor real.

### Com Validação Cruzada
- **R²: 0.833689**: Com a validação cruzada, o R² aumentou para 83.37%, indicando uma ligeira melhoria na capacidade do modelo de explicar a variância nos dados.
- **RMSE: 592832.342916**: O RMSE diminuiu para 592832.342916, o que significa que as previsões do modelo estão, em média, mais próximas dos valores reais em comparação com o modelo sem validação cruzada.

### Interpretação dos Resultados
A validação cruzada ajudou a melhorar o desempenho do modelo, resultando em um R² ligeiramente maior e um RMSE menor. Isso sugere que o modelo com validação cruzada é mais robusto e tem melhor capacidade preditiva, reduzindo o risco de overfitting e fornecendo previsões mais precisas.

# Preparação para sistema de recomendação

É importante padronizarmos os dados para conseguirmos utilizar o PCA

In [14]:
# 1. Criação e Treinamento do Scaler
scaler = StandardScaler(inputCol='features', outputCol='scaledFeatures', withStd=True, withMean=False)
scaler_modelo = scaler.fit(modelo_vectorizado)

# 2. Aplicação do Scaler nos Dados
modelo_padronizado = scaler_modelo.transform(modelo_vectorizado)

# 3. Seleção das Colunas Relevantes
modelo_padronizado = modelo_padronizado.select('scaledFeatures', 'valor')

1. **Criação e Treinamento do Scaler**:
   - O **StandardScaler** é uma ferramenta para **padronizar** as features de um conjunto de dados. Ele ajusta as variáveis para que tenham um desvio padrão de 1, o que é importante quando variáveis têm escalas diferentes. A padronização ajuda a garantir que todas as variáveis contribuam igualmente para o modelo, evitando que variáveis com maior escala dominem o processo de aprendizado.
   - No código, a transformação é aplicada à coluna `features` e os resultados escalados são armazenados na coluna `scaledFeatures`.
   - O método **fit()** calcula os parâmetros necessários para a padronização (como o desvio padrão) com base nas variáveis presentes na coluna `features` do DataFrame `modelo_vectorizado`. Este é um passo de aprendizado, onde o modelo calcula como as variáveis devem ser transformadas.

2. **Aplicação do Scaler nos Dados**:
   - O método **transform()** aplica a transformação calculada pelo **fit()** aos dados de entrada, gerando uma nova coluna (`scaledFeatures`) com as features já escaladas. As variáveis originais na coluna `features` são transformadas para a mesma escala, o que torna os dados mais consistentes e equilibrados para o modelo de aprendizado.

3. **Seleção das Colunas Relevantes**:
   - Após a transformação, o código seleciona apenas as colunas `scaledFeatures` e `valor`, que representam, respectivamente, as variáveis preditoras escaladas e a variável alvo (valor). O restante das colunas é descartado para focar apenas nas variáveis relevantes para o modelo.

**Importância dessa Etapa:**
- A padronização é um pré-requisito para garantir que o PCA opere de maneira justa e equilibrada entre as diferentes variáveis, resultando em uma melhor redução de dimensionalidade e em componentes principais que refletem a verdadeira variação dos dados.

Para conseguirmos criar nosso modelo de recomendação, precisamos reduzir a dimensão dos nossos dados. Para fazermos isso, podemos utilizar a técnica chamada PCA.

In [15]:
# 1. Configuração do PCA
# Número de K = número de features
pca = PCA(k=22, inputCol='scaledFeatures', outputCol='pca_features')

# 2. Ajuste do Modelo PCA
modelo = pca.fit(modelo_padronizado)

# 3. Aplicação do Modelo PCA nos Dados
modelo_pca = modelo.transform(modelo_padronizado)

1. **Configuração do PCA**:
   - **PCA (Principal Component Analysis)** é uma técnica de redução de dimensionalidade que transforma um conjunto de variáveis correlacionadas em um conjunto de variáveis não correlacionadas, chamadas de componentes principais. O objetivo do PCA é preservar a maior parte da variação dos dados enquanto reduz o número de variáveis.
   - O PCA é configurado para:
     - **k=22**: Isso significa que o PCA reduzirá os dados originais para 22 componentes principais, ou seja, manterá 22 novas variáveis que representam a maior parte da variabilidade dos dados originais.
     - **inputCol="scaledFeatures"**: O PCA aplicará a transformação sobre as variáveis que já foram escaladas e armazenadas na coluna `scaledFeatures`.
     - **outputCol="pca_features"**: O resultado da transformação será armazenado na coluna `pca_features`, que conterá os 22 componentes principais calculados.

2. **Ajuste do Modelo PCA**:
   - O método **fit()** calcula os componentes principais com base nas **features escaladas** e ajusta o modelo PCA para os dados fornecidos. Esse modelo PCA agora está pronto para ser usado em dados de entrada e transformá-los nas novas variáveis (componentes principais).

3. **Aplicação do Modelo PCA nos Dados**:
   - O método transform() aplica a transformação PCA aos dados. O resultado, modelo_pca, contém os dados com as features originais reduzidas para os 22 componentes principais, que são armazenados na coluna pca_features.

**Importância dessa Etapa**:
- A transformação dos dados em **componentes principais** ajuda a eliminar **multicolinearidade** (correlação entre as variáveis), tornando os dados mais adequados para certos algoritmos de machine learning.
- Reduzindo o número de features, o PCA também pode melhorar a **interpretação** dos dados, pois as componentes principais podem capturar os aspectos mais relevantes da variabilidade dos dados de uma maneira mais compacta e gerenciável.

In [16]:
# 1. Cálculo da Variância Explicada Acumulada
lista_valores = [sum(modelo.explainedVariance[0:i+1]) for i in range(22)]

sum(np.array(lista_valores) <= 0.9) # 2. 90% é a taxa mínima de explicação que escolhi.

12

1. **Cálculo da Variância Explicada Acumulada**:
   Cria uma lista que armazena a **variância explicada acumulada**. A cada novo componente principal, a variância explicada é somada ao total acumulado, permitindo ver **quanto da variabilidade total dos dados** é capturado progressivamente pelos componentes principais.

2. **Determinação do Número de Componentes que Explicam pelo Menos 90% da Variância**:
   O último passo conta quantos componentes principais são necessários para explicar **90% da variância** dos dados. Isso ajuda a decidir quantos componentes manter para uma boa redução de dimensionalidade, garantindo que a maior parte da informação dos dados seja preservada ao reduzir o número de variáveis (componentes principais).

**Importância dessa etapa**:
- Esse resultado significa que, ao usar os primeiros 12 componentes, você consegue reduzir a dimensionalidade dos dados mantendo uma quantidade significativa de informação (90% da variabilidade dos dados), o que pode melhorar a eficiência computacional e reduzir o risco de overfitting, sem perder muito poder preditivo no modelo.

Usando o método do cotovelo para descobrir o melhor número de K's na clusterização

In [17]:
# Inicializar uma lista para armazenar os valores de SSE
sse_values = {}

# Testar diferentes valores de K (número de clusters)
for k in range(2, 51):  # de 2 até 50 clusters
    kmeans = KMeans(k=k, seed=1, featuresCol='pca_features', predictionCol='cluster')
    model = kmeans.fit(modelo_pca)

    # Predições e SSE
    predictions = model.transform(modelo_pca)
    evaluator = ClusteringEvaluator(predictionCol='cluster', featuresCol='pca_features')
    sse = evaluator.evaluate(predictions)  # Calcula a medida de erro (SSE)

    sse_values[k] = sse

A **importância dessa etapa** é a seguinte:

1. **Avaliação de Diferentes Quantidades de Clusters**:
   O código testa diferentes valores de **`k`** (número de clusters) para o modelo K-means. Isso é importante porque o número de clusters ideal não é algo que se sabe de antemão e precisa ser determinado com base nos dados. A ideia é explorar vários valores de `k` para entender como o número de clusters afeta a qualidade do modelo.

2. **Cálculo do SSE (Soma dos Erros Quadráticos)**:
   A **Soma dos Erros Quadráticos (SSE)** é uma métrica usada para avaliar a qualidade do clustering. Ela mede a soma das distâncias quadradas entre os pontos de dados e os centros dos clusters. Quanto menor o SSE, melhor o modelo de clustering, pois significa que os pontos de dados estão mais próximos dos centros dos clusters atribuídos. A importância dessa métrica é fornecer uma indicação de quão bem o modelo está agrupando os dados. Ao comparar o SSE para diferentes valores de `k`, é possível identificar um número de clusters que minimize o erro, equilibrando a complexidade do modelo e a qualidade do agrupamento.

3. **Exploração de Diferentes Valores de K (Número de Clusters)**:
   O código percorre valores de **`k`** de 2 até 50 e calcula o SSE para cada um desses valores. Isso é crucial para entender como o modelo se comporta com diferentes números de clusters. A principal vantagem disso é que podemos buscar por um **número ótimo de clusters**, geralmente identificado pelo ponto de inflexão na curva do SSE. Esse ponto, conhecido como "cotovelo", indica onde adicionar mais clusters não resulta em uma melhoria significativa no modelo.

4. **Armazenamento dos Resultados de SSE**:
   Ao armazenar os valores de SSE para cada número de clusters `k`, o código fornece um histórico completo da avaliação do modelo, o que facilita a análise posterior. A partir desses resultados, podemos gerar uma **curva de SSE** versus o número de clusters e observar qual valor de `k` oferece o melhor equilíbrio entre precisão (baixo SSE) e complexidade (número de clusters). Isso ajuda na decisão de qual modelo de clustering utilizar, evitando tanto o subajuste (k muito baixo) quanto o sobreajuste (k muito alto).

**Importância dessa etapa**:
- Em resumo, essa etapa é essencial para **determinar o número ideal de clusters** para o problema de clustering e garantir que o modelo K-means seja ajustado corretamente, capturando a estrutura subjacente dos dados de maneira eficiente.

In [18]:
sse_values

{2: 0.3895206818434673,
 3: 0.3664486523831662,
 4: 0.3020907243729316,
 5: 0.3123389269670627,
 6: 0.3014052889671764,
 7: 0.26001675258937396,
 8: 0.26724807089138825,
 9: 0.3341900296915097,
 10: 0.33271972591770227,
 11: 0.31831104740506616,
 12: 0.33232176550573017,
 13: 0.35158795896493983,
 14: 0.29938567551563333,
 15: 0.3634638245555099,
 16: 0.34165577113809414,
 17: 0.3471505486629403,
 18: 0.3337034616551103,
 19: 0.3006307665446842,
 20: 0.3079413347937255,
 21: 0.319199921241598,
 22: 0.32378311173650254,
 23: 0.34885149442479607,
 24: 0.2954915465802582,
 25: 0.323715876569969,
 26: 0.29044913293730296,
 27: 0.30992555760292956,
 28: 0.3243813352669043,
 29: 0.3090923539518456,
 30: 0.31446167341446163,
 31: 0.255619095496541,
 32: 0.2917938800102782,
 33: 0.31510443823316653,
 34: 0.318100479729942,
 35: 0.31189712390681346,
 36: 0.31209458374246096,
 37: 0.30275981496343735,
 38: 0.2774835259051263,
 39: 0.3120397864340143,
 40: 0.32351067493508506,
 41: 0.318048554360

K = 15 foi o melhor

Utilizando Pipeline

In [19]:
dados = tratar_dados(df.select('anuncio.*'), 'rec')

pipeline = Pipeline(stages=[VectorAssembler(inputCols=features_col, outputCol='features'), StandardScaler(inputCol='features', outputCol='scaledFeatures', withStd=True, withMean=False), PCA(k=12, inputCol='scaledFeatures', outputCol='pca_features'), KMeans(k=15, featuresCol='pca_features', predictionCol='cluster')])
pipeline_modelo = pipeline.fit(dados)
df_pronto = pipeline_modelo.transform(dados)

Criação de pipeline com os melhores parametros encontrados, cada componente no pipeline tem uma função específica (Para reforçar):

1. **VectorAssembler**: Prepara os dados, combinando várias colunas de características em um único vetor. Isso é essencial para transformar os dados em um formato adequado para os algoritmos de aprendizado de máquina, como KMeans.

2. **StandardScaler**: Normaliza as características, garantindo que todas as variáveis tenham a mesma escala. Isso é importante porque muitos algoritmos, como o PCA e KMeans, são sensíveis à escala dos dados, e a padronização ajuda a melhorar a performance do modelo.

3. **PCA (Principal Component Analysis)**: Reduz a dimensionalidade dos dados, mantendo as componentes mais significativas. Isso ajuda a diminuir a complexidade do modelo e melhora a eficiência computacional, especialmente em grandes conjuntos de dados.

4. **KMeans**: Realiza o agrupamento dos dados em clusters, com base nas características transformadas. O modelo tenta identificar grupos semelhantes de dados, o que pode ser útil para várias análises, como segmentação de clientes ou identificação de padrões.

**Importância dessa etapa**:
- Ao unir essas etapas em um pipeline, o processo se torna mais eficiente e repetível, eliminando a necessidade de realizar cada transformação separadamente. Além disso, a utilização de um pipeline facilita ajustes futuros no modelo, já que todas as etapas são encapsuladas de forma organizada.

# Função para Sistema Recomendador

In [22]:
def recomenda_imovel(id, df):

  # Função da distância euclidiana
  def distancia(imovel, valor):
    return euclidean(imovel, valor)

  # 1. Identificação do cluster
  cluster = df\
        .filter(df.id == id)\
        .select('cluster')\
        .collect()[0][0]

  # 2. Filtragem de imóveis no mesmo cluster
  imoveis_recomendados = df\
      .filter(df.cluster == cluster)

  # 3. Obtendo as características PCA do imóvel procurado
  imovel_procurado = imoveis_recomendados\
      .filter(imoveis_recomendados.id == id)\
      .select('pca_features')\
      .collect()[0][0]

  # 4. Cálculo da distância euclidiana
  distancia_udf = f.udf(lambda x: distancia(
      imovel_procurado, x), FloatType())

  # 5. Ordenação e filtragem dos imóveis
  colunas_nao_utilizadas = [
      'features', 'scaled_features', 'pca_features', 'cluster', 'distancia']

  recomendacao = imoveis_recomendados\
      .withColumn('distancia', distancia_udf('pca_features'))\
      .filter(imoveis_recomendados.id != id)\
      .select([col for col in imoveis_recomendados.columns if col not in colunas_nao_utilizadas])\
      .orderBy('distancia')

  # 6. Retorno das recomendações
  return recomendacao.select('andar', 'area_util', 'quartos', 'suites', 'vaga', 'bairro', 'valor').show(10, truncate=False)

A função **`recomenda_imovel`** tem como objetivo recomendar imóveis semelhantes a um imóvel específico com base em seu cluster e utilizando a **distância euclidiana** entre as características dos imóveis, após a aplicação do **PCA (Principal Component Analysis)** para redução de dimensionalidade. A função segue estas etapas:

1. **Identificação do cluster**: A partir do ID do imóvel fornecido, a função obtém o cluster ao qual o imóvel pertence. Isso é feito para garantir que a recomendação seja feita dentro do mesmo grupo de imóveis, baseando-se em características semelhantes.

2. **Filtragem de imóveis no mesmo cluster**: Após identificar o cluster, a função seleciona todos os imóveis que pertencem ao mesmo grupo, aumentando a chance de recomendação de imóveis realmente similares.

3. **Obtendo as características PCA do imóvel procurado**: A função coleta as características PCA do imóvel que foi fornecido como entrada, que representam as variáveis mais significativas do imóvel no espaço de componentes principais.

4. **Cálculo da distância euclidiana**: A função define uma métrica de distância (distância euclidiana) entre o imóvel procurado e outros imóveis, utilizando as características PCA. Essa distância é usada para avaliar a similaridade entre os imóveis.

5. **Ordenação e filtragem dos imóveis**: Com base na distância calculada, a função ordena os imóveis do cluster, colocando os mais semelhantes ao imóvel procurado no topo da lista. Algumas colunas desnecessárias (como as características PCA e o próprio cluster) são removidas para limpar o resultado.

6. **Retorno das recomendações**: A função retorna o conjunto de imóveis recomendados, ordenados pela sua similaridade com o imóvel procurado, com base na distância euclidiana.

In [23]:
recomenda_imovel('0034df72-124a-4383-a89f-a019850a2ba0', df_pronto)

+-----+---------+-------+------+----+------------------------+---------+
|andar|area_util|quartos|suites|vaga|bairro                  |valor    |
+-----+---------+-------+------+----+------------------------+---------+
|0    |140.0    |3      |2     |2   |Recreio dos Bandeirantes|887000.0 |
|0    |228.0    |3      |1     |2   |Barra da Tijuca         |1885000.0|
|0    |228.0    |3      |1     |2   |Barra da Tijuca         |1882000.0|
|0    |298.0    |2      |2     |1   |Barra da Tijuca         |4500000.0|
|0    |115.0    |3      |2     |2   |Barra da Tijuca         |1498000.0|
|0    |115.0    |3      |2     |2   |Barra da Tijuca         |1497000.0|
|0    |148.0    |3      |1     |2   |Recreio dos Bandeirantes|1198000.0|
|0    |148.0    |3      |1     |2   |Recreio dos Bandeirantes|1199000.0|
|0    |148.0    |3      |1     |2   |Recreio dos Bandeirantes|1200000.0|
|0    |140.0    |3      |1     |2   |Recreio dos Bandeirantes|888000.0 |
+-----+---------+-------+------+----+--------------